In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import re

In [ ]:
included_studies = []

with open("final_processed_studies.jsonl", "r") as f:
    for line in f:
        included_studies.append(json.loads(line.strip()))

# Quickly Regenerate the Most Specific Dates
The reason we must do this is because some studies are duplicated across databases (which we removed in the deduplication step), but some databases have more specific dates than others. We want the most specific date possible for our analysis, so we must go through each database and try to find them.

In [ ]:
# we're going to open up all the database files to try our hardest to match a date to each study if it doesn't have one
import pandas as pd
from rapidfuzz import fuzz
from tqdm import tqdm
import json

def get_year(row):
    for key in ["Publication Year", "Year"]:
        if key in row and pd.notnull(row[key]):
            try:
                return int(row[key])
            except ValueError:
                return None
    return None

unique_id_cols = {
    "PMID",
    "PMCID",
    "NIHMS ID",
    "DOI",
    "EID",
    "Clinical Trial Numbers"
}

embase = pd.read_csv("embase-export-9-6-25.csv", sep=",").to_dict(orient='records')
pubmed = pd.read_csv("pubmed-export-9-6-25.csv", sep=",").to_dict(orient='records')
scopus = pd.read_csv("scopus-export-9-6-25.csv", sep=",").to_dict(orient='records')

import re
from datetime import datetime

# define the cutoff once
_CUTOFF = datetime(2025, 9, 6)

def find_most_specific_date(title, return_raw_candidates=False):
    # Gather all date‐strings from the three sources
    candidates = []
    for item in embase:
        if title.lower() in str(item.get("Title", "")).lower():
            candidates.append(item.get("Date of Publication"))
    for item in pubmed:
        if title.lower() in str(item.get("Title", "")).lower():
            candidates.append(item.get("Create Date"))
    for item in scopus:
        if title.lower() in str(item.get("Title", "")).lower():
            candidates.append(item.get("Year"))
    
    if return_raw_candidates:
        return candidates
    
    # Define the formats and their specificity levels
    formats = [
        ('%d %b %Y', 3),   # e.g. '9 Jul 2023'
        ('%d %B %Y', 3),   # e.g. '9 July 2023'
        ('%m/%d/%y', 3),   # e.g. '9/1/23'
        ('%m/%d/%Y', 3),   # e.g. '9/1/2023'
        ('%b %Y',    2),   # e.g. 'Jul 2023'
        ('%B %Y',    2),   # e.g. 'July 2023'
        ('%Y',       1),   # e.g. '2023'
    ]
    
    best_dt   = None
    best_spec = 0
    
    # Try parsing each candidate
    for raw in candidates:
        if not raw:
            continue
        s = str(raw).strip()
        parsed = False
        
        for fmt, spec in formats:
            try:
                dt = datetime.strptime(s, fmt)
                parsed = True
                break
            except ValueError:
                continue
        
        if not parsed:
            # skip anything we don’t recognize
            continue

        # given that we pulled this data before this cutoff date, we can be confident that any date in the future is incorrect
        if dt > _CUTOFF:
            continue
        
        # Keep the one with highest specificity, tiebreaker = most recent
        if spec > best_spec or (
           spec == best_spec and (best_dt is None or dt > best_dt)
        ):
            best_spec = spec
            best_dt   = dt
    
    return best_dt

for study in tqdm(included_studies):
    date = find_most_specific_date(study["Title"])
    study["Date"] = date

with open("final_processed_studies_dated.jsonl", "w") as f:
    for study in included_studies:
        f.write(json.dumps(study, default=str) + "\n")

In [ ]:
with open("final_processed_studies_dated.jsonl", "r") as f:
    included_studies = [json.loads(line.strip()) for line in f]

# Get Model Counts

In [ ]:
model_counts = {}
for study in included_studies:
    for model in study["processed_data"]["model_categories"]:
        if model == "":
            model_counts["Unknown"] = model_counts.get("Unknown", 0) + 1
        if model not in model_counts:
            model_counts[model] = 0
        
        # We only include "Other" if it is the only model category
        if "Other" in study["processed_data"]["model_categories"] and len(study["processed_data"]["model_categories"]) == 1 and model == "Other":
            model_counts["Other"] += 1
        elif model != "Other":
            model_counts[model] += 1

sorted_model_counts = sorted(model_counts.items(), key=lambda x: x[1], reverse=True)

# Determine the longest model name
max_model_length = max(len(model) for model, _ in sorted_model_counts)

# get win rate against humans based on model name, level of training, and tier
win_rates = {}

# Optional header
print(f"{'Model'.ljust(max_model_length)} | Studies")
print('-' * (max_model_length + 10))

# Print formatted model counts
for model, count in sorted_model_counts:
    print(f"{model.ljust(max_model_length)} | {str(count).rjust(7)}")

# sum total studies
total_studies = sum(count for _, count in sorted_model_counts)
print(f"\nTotal studies: {total_studies}")

In [ ]:
(515+302+128+21+23+6)/7623

In [ ]:
def compute_open_vs_closed(sorted_model_counts):
    # Define which models are open source
    open_source_models = {
        "LLaMA", "LLaMA 2", "LLaMA 3.x",
        "Mistral", "Mixtral", "Gemma", "Qwen", "GLM / ChatGLM",
        "LLaVA", "Baichuan", "Vicuna", "WizardLM", "BLOOM",
        "Falcon", "Yi", "Nous Hermes", "MPT", "Zephyr",
        "DeepSeek", "MiniCPM"
    }

    open_count = 0
    closed_count = 0
    unknown_count = 0

    for model, count in sorted_model_counts:
        if model in open_source_models:
            open_count += count
        elif model == "Other" or model == "Unknown":
            unknown_count += count
        else:
            closed_count += count

    return {
        "open_source": open_count,
        "closed_source": closed_count,
        "unknown": unknown_count,
        "total": open_count + closed_count + unknown_count
    }

results = compute_open_vs_closed(sorted_model_counts)
print(results)

# Get Tasks

In [ ]:
task_types_counts = {}
num_tasks = []  # Make sure this list is initialized

for study in included_studies:
    if len(study["processed_data"]["task_types"]) == 0:
        task_types_counts["Unknown"] = task_types_counts.get("Unknown", 0) + 1
    num_tasks.append(len(study["processed_data"]["task_types"]))
    for task in study["processed_data"]["task_types"]:
        if task not in task_types_counts:
            task_types_counts[task] = 0
        task_types_counts[task] += 1

sorted_task_types_counts = sorted(task_types_counts.items(), key=lambda x: x[1], reverse=True)

# Determine the longest task type name
max_task_length = max(len(task) for task, _ in sorted_task_types_counts)

# Compute total for percentage calculation
total_studies = sum(task_types_counts.values())

# Optional header
print(f"{'Task Type'.ljust(max_task_length)} | Studies | Percent")
print('-' * (max_task_length + 22))

# Print formatted task types counts with percentages
for task, count in sorted_task_types_counts:
    percentage = (count / total_studies) * 100
    print(f"{task.ljust(max_task_length)} | {str(count).rjust(7)} | {percentage:6.2f}%")

# Get Dataset Types

In [ ]:
datasets_used_counts = {}
studies_that_have_datasets = 0
num_types = {}
num_datasets = []
for study in included_studies:
    found = False

    for item in study["processed_data"]["dataset_types"]:
        category = item.get("category", "Unknown")
        access_type = item.get("access", "unsure")

        if category not in datasets_used_counts:
            datasets_used_counts[category] = 0
        datasets_used_counts[category] += 1

        if access_type not in num_types:
            num_types[access_type] = 0
        num_types[access_type] += 1

        found = True
    
    num_datasets.append(len(study["processed_data"]["dataset_types"]))

    if found:
        studies_that_have_datasets += 1
    else:
        datasets_used_counts["Undefined in Abstract"] = datasets_used_counts.get("Undefined in Abstract", 0) + 1
        

sorted_datasets_used_counts = sorted(datasets_used_counts.items(), key=lambda x: x[1], reverse=True)

# Determine the longest dataset name
max_dataset_length = max(len(dataset) for dataset, _ in sorted_datasets_used_counts)
# Optional header
print(f"{'Dataset'.ljust(max_dataset_length)} | Studies | %")
print('-' * (max_dataset_length + 10))

# Print formatted datasets used counts
for dataset, count in sorted_datasets_used_counts:
    print(f"{dataset.ljust(max_dataset_length)} | {str(count).rjust(7)} | {count / len(included_studies) * 100:.2f}%")

print("\n\n")

# Print dataset access types
print("Dataset Access Types:")
for access_type, count in num_types.items():
    print(f"{access_type.ljust(12)} : {str(count).rjust(3)}")

# print total studies that used datasets
print(f"\nTotal studies that used datasets: {studies_that_have_datasets} out of {len(included_studies)}")
print(f"Total number of datasets used: {sum(datasets_used_counts.values())}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams.update({
    'font.family': 'serif',
    'axes.titlesize': 12,
    'axes.labelsize': 11,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.spines.left': True,
    'axes.spines.bottom': True,
    'axes.linewidth': 1.0,
    'figure.facecolor': 'white',
})

labels = [
    "Clinician Board & Self-Assessment Questions",
    "Patient-Facing Q&A & FAQs",
    "Clinical Vignettes & Case Reports",
    "Clinical Guidelines & Consensus Statements",
    "Imaging Data & Reports",
    "Real-World Electronic Health Records",
    "Survey Instruments & Psychometric Scales",
    "Educational & Reference Texts",
    "Dialogue & Chat Corpora",
    "Synthetic / Simulated Clinical Data",
    "Social-Media & Search-Query Data",
    "Research Literature & Abstract Corpora",
    "Other",
    "Regulatory & Administrative Documents",
    "Pharmacology & Medication Knowledge",
    "Physiological Signals & Wearables",
    "Laboratory & Pathology Data",
    "Knowledge Graphs / Ontologies / Terminologies",
    "Clinical Trial Eligibility & Structured Cohorts",
    "Imaging Challenges & Benchmarks",
    "Genomics & -Omics",
    "Benchmark evaluation items",
]

counts = [
    1047, 691, 652, 579, 472, 423, 394, 307, 235, 194, 178, 142,
    87, 68, 62, 61, 57, 51, 41, 38, 30, 1
]

# sort descending
order = np.argsort(counts)[::-1]
labels_sorted = [labels[i] for i in order]
counts_sorted = [counts[i] for i in order]
y = np.arange(len(labels_sorted))

# alternating greys
color1, color2 = '0.2', '0.4'
bar_colors = [color1 if i % 2 == 0 else color2 for i in range(len(labels_sorted))]

# --- plot ---
fig, ax = plt.subplots(figsize=(10, 5))
ax.barh(y, counts_sorted, height=0.7, color=bar_colors)
ax.set_yticks(y)
ax.set_yticklabels(labels_sorted)
for tick, c in zip(ax.get_yticklabels(), bar_colors):
    tick.set_color(c)

ax.invert_yaxis()  # biggest at top

# value labels to the right of each bar
pad = max(counts_sorted) * 0.01 + 6
for i, v in enumerate(counts_sorted):
    ax.text(v + pad, i, f"{v}", va='center', fontsize=9)

ax.set_xlabel("Number of Studies")
ax.set_title("Study Dataset Types", pad=10)

# light grid on x to match style
ax.xaxis.grid(True, linestyle='-', linewidth=0.3, color='0.85')
ax.set_axisbelow(True)

plt.tight_layout()
plt.savefig("study_dataset_types.svg", dpi=300)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams.update({
    'font.family': 'serif',
    'axes.titlesize': 12,
    'axes.labelsize': 11,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.spines.left': True,
    'axes.spines.bottom': True,
    'axes.linewidth': 1.0,
    'figure.facecolor': 'white',
})

models = [
    "ChatGPT-unspecified", "GPT-4", "GPT-4o", "ChatGPT-4", "GPT-3.5", "Gemini",
    "ChatGPT-3.5", "Claude", "Bard", "LLaMA 3.x", "Bing Chat",
    "DeepSeek", "Gemini 1.5", "LLaMA 2", "Perplexity", "Mistral", "GPT-unspecified",
    "LLaMA", "Qwen", "Other"
]

studies = [
    1292, 1252, 753, 626, 600, 515, 381, 325, 302,
    182, 167, 132, 128, 95, 87, 70, 70, 59, 57, 530
] # Pulled from the above tables!

# Sort descending
order = np.argsort(studies)[::-1]
sorted_models = [models[i] for i in order]
sorted_studies = [studies[i] for i in order]
y_pos = np.arange(len(sorted_models))

# Alternating subtle greys
color1 = '0.2'
color2 = '0.4'
bar_colors = [color1 if i % 2 == 0 else color2 for i in range(len(sorted_models))]

# Plot
fig, ax = plt.subplots(figsize=(6, 6))

# Spread bars: height=0.7
ax.barh(y_pos, sorted_studies, color=bar_colors, height=0.7)
ax.set_yticks(y_pos)
ax.set_yticklabels(sorted_models)

# Alternate y-tick label colors to match bars
for tick_label, color in zip(ax.get_yticklabels(), bar_colors):
    tick_label.set_color(color)

ax.invert_yaxis()  # top-down

# Annotate bar values
for i, v in enumerate(sorted_studies):
    ax.text(v + 10, i, f"{v}", va='center', fontsize=9)

ax.set_xlabel('Number of Studies')
ax.set_title('LLM Usage in Studies', pad=10)

plt.tight_layout()
# save at 300 dpi as an svg
plt.savefig('llm_usage_studies.svg', dpi=300)
plt.show()


# Get Breakdown by Specialty

In [ ]:
specialty_counts = {}
specialty_tier_counts = {}
total_vals = 0
num_terms = []
subspecialties_only = False
broad_specialties_only = True

assert not (subspecialties_only and broad_specialties_only), "Cannot set both subspecialties_only and broad_specialties_only to True"

for study in included_studies:
    total_vals += len(study["processed_data"]["specialties"])
    for i, specialty in enumerate(study["processed_data"]["specialties"]):
        num_terms.append(len(specialty))
        if len(specialty) < 2 and subspecialties_only:
            continue
        if specialty[-1] == "Internal Medicine" and subspecialties_only:
            continue
        if subspecialties_only:
            list_to_process = specialty[-1:]
        elif broad_specialties_only:
            list_to_process = specialty[:1]
        else:
            list_to_process = specialty
        for subspecialty in list_to_process:
            if subspecialty == "Neurosurgery":
                subspecialty = "Neurological Surgery"
            if subspecialty == "Dentistry":
                subspecialty = "Dentistry and Oral and Maxillofacial Surgery"
            if subspecialty == "Surgery (American Board of Surgery)":
                subspecialty = "General Surgery"

            if subspecialty == "Radiology":
                subspecialty = "Interventional Radiology and Diagnostic Radiology"

            if subspecialty == "Public Health and General Preventive Medicine":
                subspecialty = "Preventive Medicine"

            if subspecialty == "Neurology (Adult)":
                subspecialty = "Neurology"

            if subspecialty == "Pediatrics (General)":
                subspecialty = "Pediatrics"

            if subspecialty == "Hospitals and Palliative Medicine":
                subspecialty = "Hospice and Palliative Medicine"

            if subspecialty == "Psychiatry and Neurology (Shared/Multi-board subs)":
                subspecialty = "Neuropsychiatry"

            if subspecialty == "Pediatric Orthopedics" or subspecialty == "Pediatric Orthopaediology" or subspecialty == "Pediatric Orthopaedic Surgery":
                subspecialty = "Pediatric Orthopaedics"

            if subspecialty == "Pathology - Medical Microbiology" or subspecialty == "Medical Microbiology Pathology":
                subspecialty = "Medical Microbiology (Pathology)"

            if subspecialty == "Urogynecology and Reconstructive Pelvic Surgery":
                subspecialty = "Female Pelvic Medicine and Reconstructive Surgery"

            if subspecialty == "Child Neurology":
                subspecialty = "Pediatric Neurology"

            if subspecialty == "Pathology":
                subspecialty = "Pathology - Anatomic/Clinical (AP/CP)"

            if subspecialty == "Internal Medicine-Critical Care Medicine":
                subspecialty = "Critical Care Medicine"
            
            if subspecialty == "Pathology - Anatomic/Clinical (AP/CP)":
                subspecialty = "Pathology"

            if subspecialty == "Otolaryngology - Head and Neck Surgery":
                subspecialty = "Otolaryngology"

            # now use `subspecialty` downstream


            if subspecialty not in specialty_counts:
                specialty_counts[subspecialty] = 0

            if subspecialty == "":
                subspecialty = "Unknown"
    
            if subspecialty not in specialty_tier_counts:
                specialty_tier_counts[subspecialty] = {
                    "S": 0,
                    "I": 0,
                    "II": 0,
                    "III": 0,
                }

            tier = study["LLM-tier"]["Tier"]
            specialty_tier_counts[subspecialty][tier] += 1
            
            specialty_counts[subspecialty] += 1
            #break

# Determine the longest specialty name for formatting
max_specialty_length = max(len(s) for s in specialty_counts.keys())

# Header (optional)
print(f"{'Specialty'.ljust(max_specialty_length)} | {'Total':>5} | Tier S | Tier I | Tier II | Tier III")
print('-' * (max_specialty_length + 50))

# sort specialties by count
specialty_counts = dict(sorted(specialty_counts.items(), key=lambda item: item[1], reverse=True))

# Print formatted rows
residual = 0
residual_count = {"S": 0, "I": 0, "II": 0, "III": 0}
for specialty, count in specialty_counts.items():
    tiers = specialty_tier_counts[specialty]
    if count/len(included_studies)*100 > 0.21:
        print(f"{specialty.ljust(max_specialty_length)} | {(str(count/len(included_studies)*100)[:4] + "%").rjust(5)} | "
            f"{str(tiers['S']).rjust(6)} | {str(tiers['I']).rjust(6)} | "
            f"{str(tiers['II']).rjust(7)} | {str(tiers['III']).rjust(8)} | {tiers['S'] + tiers['I'] + tiers['II'] + tiers['III']}")
    else:
        residual += count/len(included_studies)*100
        residual_count["S"] += tiers["S"]
        residual_count["I"] += tiers["I"]
        residual_count["II"] += tiers["II"]
        residual_count["III"] += tiers["III"]

print(f"Residual: {residual:.2f}% | Count: {residual_count}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mtick

mpl.rcParams.update({
    'font.family': 'serif',
    'axes.titlesize': 12,
    'axes.labelsize': 11,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.spines.left': True,
    'axes.spines.bottom': True,
    'axes.linewidth': 1.0,
    'figure.facecolor': 'white',
})

total = len(included_studies)

spec_df = (pd.DataFrame({
    'Specialty': list(specialty_counts.keys()),
    'Count':     list(specialty_counts.values())
})
  .assign(Percent=lambda df: df['Count']/total)
  .sort_values('Count', ascending=False)
  .reset_index(drop=True)
)

labels = spec_df['Specialty']
percent = spec_df['Percent']
y_pos = np.arange(len(labels))

# include only the first N specialties to avoid clutter
N = 30
if len(labels) > N:
    labels = labels[:N]
    percent = percent[:N]
    y_pos = y_pos[:N]

# alternating greys
color1 = '0.2'
color2 = '0.4'
bar_colors = [color1 if i%2==0 else color2 for i in range(len(labels))]

fig, ax = plt.subplots(figsize=(6, 8))
ax.barh(y_pos, percent, color=bar_colors, height=0.7)
ax.xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))

# invert so highest on top
ax.invert_yaxis()
# Set y-tick labels with alternating colors
ax.set_yticks(y_pos)
for i, label in enumerate(labels):
    tick_color = color1 if i % 2 == 0 else color2
    ax.text(-0.01, i, label, va='center', ha='right', fontsize=10, color=tick_color,
            transform=ax.get_yaxis_transform(), clip_on=False)
ax.set_yticklabels(['' for _ in labels])  # Hide default ticks

ax.set_xlabel('Percentage of Studies')
ax.set_title('Study Distribution by Specialty')

# annotate percentages
for i, (p, cnt) in enumerate(zip(percent, spec_df['Count'])):
    ax.text(p + 0.01, i, f"{p:.1%} (n={cnt})", va='center', fontsize=9)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd

labels = spec_df['Specialty']
percent = spec_df['Percent']
x_pos = np.arange(len(labels))

# include only the first N specialties to avoid clutter
N = 22
if len(labels) > N:
    labels = labels[:N]
    percent = percent[:N]
    x_pos = x_pos[:N]

# Set alternating colors
color1 = '0.2'
color2 = '0.4'
bar_colors = [color1 if i % 2 == 0 else color2 for i in range(len(labels))]

# ── Plot ──
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(x_pos, percent, color=bar_colors, width=0.7)

ax.set_ylabel('Percentage of Studies')
ax.set_title('Proportion of Studies by Specialty (first subspecialty)')
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))

# Annotate bars with percentages
for i, (p, cnt) in enumerate(zip(percent, spec_df['Count'])):
    ax.text(i, p + 0.01, f"{p:.1%}", ha='center', va='bottom', fontsize=9)

# Alternating tick label colors
ax.set_xticks(x_pos)
ax.set_xticklabels(labels, rotation=45, ha='right')
for tick_label, i in zip(ax.get_xticklabels(), range(len(labels))):
    tick_label.set_color(color1 if i % 2 == 0 else color2)

plt.tight_layout()
# save at 300 dpi as an svg
plt.savefig('study_specialties_bar_chart.svg', dpi=300)
plt.show()


In [ ]:
# Run the get specialty counts code first with these flags:
# subspecialties_only = True
# broad_specialties_only = False

specialty_counts = {}
specialty_tier_counts = {}
total_vals = 0
detected_studies = 0
num_terms = []

ABMS_SURGICAL_SPECIALTIES = [
    "Colon and Rectal Surgery",
    "Neurological Surgery",
    "Obstetrics and Gynecology",
    "Opthalmology",
    "Orthopaedic Surgery",
    "Otolaryngology - Head and Neck Surgery",
    "Plastic Surgery",
    "General Surgery",
    "Vascular Surgery",
    "Thoracic Surgery",
    "Urology",
]

ABIM_IM_SPECIALTIES = [
    "Adolescent Medicine",
    "Adult Congenital Heart Disease",
    "Advanced Heart Failure and Transplant Cardiology",
    "Cardiovascular Disease",
    "Clinical Cardiac Electrophysiology",
    "Critical Care Medicine",
    "Endocrinology, Diabetes and Metabolism",
    "Gastroenterology",
    "Geriatric Medicine",
    "Hematology",
    "Hospice and Palliative Medicine",
    "Infectious Disease",
    "Interventional Cardiology",
    "Medical Oncology",
    "Nephrology",
    "Neurocritical Care",
    "Pulmonary Disease",
    "Rheumatology",
    "Sleep Medicine",
    "Sports Medicine",
    "Transplant Hepatology"
]

categories_of_interest = ABIM_IM_SPECIALTIES


for study in included_studies:
    found = False
    total_vals += len(study["processed_data"]["specialties"])
    for i, specialty in enumerate(study["processed_data"]["specialties"]):
        num_terms.append(len(specialty))
        if not any(cat in specialty for cat in categories_of_interest):
            continue
        for subspecialty in specialty:
            if subspecialty not in categories_of_interest:
                continue
            found = True
            #if "surgery" in specialty[0].lower() and "surgery" not in subspecialty.lower():
            #    subspecialty = "Surgery: " + subspecialty
            
            if subspecialty not in specialty_counts:
                specialty_counts[subspecialty] = 0

            if subspecialty == "":
                subspecialty = "Unknown"
    
            if subspecialty not in specialty_tier_counts:
                specialty_tier_counts[subspecialty] = {
                    "S": 0,
                    "I": 0,
                    "II": 0,
                    "III": 0,
                }

            tier = study["LLM-tier"]["Tier"]
            specialty_tier_counts[subspecialty][tier] += 1
            
            specialty_counts[subspecialty] += 1
        
    if found:
        detected_studies += 1

# Determine the longest specialty name for formatting
max_specialty_length = max(len(s) for s in specialty_counts.keys())

# Header (optional)
print(f"{'Specialty'.ljust(max_specialty_length)} | {'Total':>5} | Tier S | Tier I | Tier II | Tier III")
print('-' * (max_specialty_length + 50))

# sort specialties by count
specialty_counts = dict(sorted(specialty_counts.items(), key=lambda item: item[1], reverse=True))

# Print formatted rows
for specialty, count in specialty_counts.items():
    tiers = specialty_tier_counts[specialty]
    print(f"{specialty.ljust(max_specialty_length)} | {(str(count/detected_studies*100)[:5] + "%").rjust(5)} | "
          f"{str(tiers['S']).rjust(6)} | {str(tiers['I']).rjust(6)} | "
          f"{str(tiers['II']).rjust(7)} | {str(tiers['III']).rjust(8)} | {tiers['S'] + tiers['I'] + tiers['II'] + tiers['III']}")

# Sample sizes

In [ ]:
# 1) build your extractor:
NUM      = r'[≈~]?\d+(?:,\d{3})*(?:\+)?'
first_re = re.compile(rf'^\s*({NUM})')

def extract_first(s: str) -> str | None:
    """
    Return the first sample-size–like token in s (with commas, ±, trailing +),
    or None if nothing matches.
    """
    m = first_re.match(s)
    return m.group(1) if m else None

sample_sizes = []
studies_with_sample_sizes = []
for study in included_studies:
    if (raw := study["extracted_data"]["sample_size"]):
        if (num := extract_first(raw)):
            sample_sizes.append(int(num.replace('≈', '').replace(',', '').replace('+', '').replace('~', '')))
            studies_with_sample_sizes.append(study)

# get sample size by tier
sample_size_by_tier = {
    "S": [],
    "I": [],
    "II": [],
    "III": []
}
for study in included_studies:
    if (raw := study["extracted_data"]["sample_size"]):
        if (num := extract_first(raw)):
            sample_size_by_tier[study["LLM-tier"]["Tier"]].append(int(num.replace('≈', '').replace(',', '').replace('+', '').replace('~', '')))

In [ ]:
# Just to be sure, we will print some random studies and check if the extracted sample sizes match!
import random
import textwrap

random_study = random.choice(studies_with_sample_sizes)
idx = studies_with_sample_sizes.index(random_study)
print(random_study["extracted_data"]["sample_size"])
print(sample_sizes[idx])
print(random_study['Title'])
print()
print("\n".join(textwrap.wrap(random_study['Abstract'], width=80)))

In [ ]:
num_studies_per_tier = {
    "I": 0,
    "II": 0,
    "III": 0
}

for study in included_studies:
    tier = study["LLM-tier"]["Tier"]
    if tier == "S":
        num_studies_per_tier["I"] += 1 # S is counted as I due to small sample size

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

tier = "all"

if tier is not "all":
    # filter the data
    filtered = np.array([x for x in sample_size_by_tier[tier] if 0 < x <= 1000])
    n = len(sample_size_by_tier[tier])
    log_vals = np.log10(filtered)
else:
    filtered = np.array([x for x in sample_sizes if 0 < x <= 1000])
    n = len(sample_sizes)
    log_vals = np.log10(filtered)

# build the empirical CDF
log_sorted = np.sort(log_vals)
cum_percent = np.arange(1, n + 1) / n

# ── 3.  plot as a line instead of bars ─────────────────────────────────
plt.plot(log_sorted, cum_percent[:len(log_sorted)], lw=2)     # continuous line
plt.fill_between(log_sorted, cum_percent[:len(log_sorted)], alpha=0.15)  # optional shading

# ── 4.  cosmetic details (same as before) ──────────────────────────────
plt.title("Cumulative percentile (outliers above 1000 removed)")
plt.xlabel("Sample size")
plt.ylabel("Cumulative Percentile")
plt.gca().yaxis.set_major_formatter(
    plt.FuncFormatter(lambda y, _: f"{y*100:.0f}%"))

# major x-ticks
major_vals = [1, 10, 100, 1000]
plt.xticks(np.log10(major_vals), [f"{v:,}" for v in major_vals])
plt.ylim(0, 1)

# minor x-ticks
minor_vals = [d * m for d in [1, 10, 100] for m in range(2, 10)]
plt.gca().set_xticks(np.log10(minor_vals), minor=True)
plt.gca().set_xticklabels([], minor=True)
plt.gca().set_yticks(np.linspace(0, 1, 11), minor=True)

# grid
plt.grid(which='major', linestyle='-', color='black', linewidth=1.2)
plt.grid(which='minor', linestyle=':', color='black', linewidth=0.8)

plt.tight_layout()
plt.savefig(f"cumulative_percentile_{tier}.svg")
plt.show()


# Plot number of studies by year

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from cycler import cycler

# style setup
grey_palette = [
    "#333333",  # darkest
    "#666666",
    "#999999",
    "#CCCCCC",  # lightest
]
mpl.rcParams.update({
    'axes.prop_cycle': cycler('color', grey_palette),
    'figure.facecolor': 'white',
    'axes.facecolor': 'white',
    'axes.grid': True,
    'grid.color': '#cccccc',   # light grey instead of black
    'grid.alpha': 0.6,         # make them partially transparent
    'grid.linewidth': 0.8,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.spines.left': True,
    'axes.spines.bottom': True,
    'xtick.direction': 'out',
    'ytick.direction': 'out',
    'xtick.color': 'black',
    'ytick.color': 'black',
    'xtick.major.size': 4,
    'ytick.major.size': 4,
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial', 'Helvetica'],
    'axes.labelsize': 10,
    'axes.titlesize': 10,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'axes.linewidth': 1.1,
})

df = pd.DataFrame(included_studies)
df['Date'] = pd.to_datetime(df['Date'])

# cumulative total and by‐tier
monthly = (
    df.assign(year_month=lambda x: x['Date'].dt.to_period('M').dt.to_timestamp())
      .groupby('year_month').size()
      .sort_index()
)
cum_total = monthly.cumsum()
start = cum_total.ne(0).idxmax()
cum_total = cum_total.loc[start:]

tiers = ['I', 'II', 'III', 'S']
tier_cums = {}
for t in tiers:
    df_t = df[df['LLM-tier'].apply(lambda d: d['Tier'] == t)]
    mon = (
        df_t.assign(year_month=lambda x: x['Date'].dt.to_period('M').dt.to_timestamp())
            .groupby('year_month').size()
            .reindex(cum_total.index, fill_value=0)
    )
    tier_cums[t] = mon.cumsum()

# plotting
fig, ax = plt.subplots(figsize=(8, 3))

ax.stackplot(
    cum_total.index,
    [tier_cums[t].values for t in tiers],
    labels=[f'Tier {t}' for t in tiers],
    colors=grey_palette,
    edgecolor='white', linewidth=0.5,
    alpha=0.9
)

# final polish
ax.set_xlabel('Date')
ax.set_ylabel('Cumulative studies')

# readable monthly ticks
tick_dates = pd.date_range(cum_total.index.min(), cum_total.index.max(), periods=12)
ax.set_xticks(tick_dates)
ax.set_xticklabels([d.strftime('%b %Y') for d in tick_dates], rotation=45, ha='right')

ax.legend(title='LLM Tier', fontsize=8, title_fontsize=9, loc='upper left')

plt.tight_layout()
# save at 300 dpi as an svg
plt.savefig('cumulative_study_counts_by_tier.svg', dpi=300)
plt.show()

In [ ]:
import pandas as pd
from scipy.stats import binomtest
from itertools import combinations

# 1) Count studies in Tiers I, II, III
df = pd.DataFrame(included_studies)
tiers = ['S', 'I','II','III']
counts = {
    tier: df['LLM-tier'].apply(lambda d: d['Tier']==tier).sum()
    for tier in tiers
}

# 2) Prepare the pairwise comparisons
pairs = list(combinations(tiers, 2))

# 3) Run tests & collect raw p-values
results = []
for a, b in pairs:
    n_a = counts[a]
    n_b = counts[b]
    total = n_a + n_b
    actual_prop = n_a / total
    expected_prop = 0.5
    
    test = binomtest(n_a, n=total, p=expected_prop, alternative='two-sided')
    p_raw = test.pvalue
    
    results.append({
        'pair': f"{a} vs {b}",
        'n_a': n_a,
        'n_b': n_b,
        'actual_prop': actual_prop,
        'p_raw': p_raw
    })

# 4) Bonferroni correction & print
m = len(results)
print(f"Pairwise 2‑tier tests with Bonferroni correction (m={m}):\n")
for res in results:
    p_adj = min(res['p_raw'] * m, 1.0)
    print(f"{res['pair']}:")
    print(f"  Observed counts     = ({res['n_a']}, {res['n_b']})")
    print(f"  Expected proportion = {expected_prop}")
    print(f"  Actual proportion   = {res['actual_prop']}")
    print(f"  raw p‑value         = {res['p_raw']}")
    print(f"  Bonferroni p‑value  = {p_adj}\n")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.lines import Line2D

df = pd.DataFrame(included_studies)
df['Date'] = pd.to_datetime(df['Date'])
monthly = (
    df.assign(year_month=lambda x: x['Date'].dt.to_period('M').dt.to_timestamp())
      .groupby('year_month').size()
      .sort_index()
)
adj = monthly.copy()
for ts, val in monthly.items():
    if ts.month == 1 and val > 0:
        adj.loc[ts] = 0
        spread = val / 12.0
        same_year = adj.index.year == ts.year
        adj.loc[same_year] += spread

# regression
x_num = mdates.date2num(adj.index)
y = adj.values
m, b = np.polyfit(x_num, y, 1)
y_fit = m * x_num + b
r2 = np.corrcoef(y, y_fit)[0, 1]**2

# slope per month (30.4375 days)
slope_mo = m * 30.4375
# proper sign formatting for intercept
eqn = rf"$y = {slope_mo:.2f}\,( \mathrm{{studies/mo}} )\,x {b:+.1f}$"

fig, ax = plt.subplots(figsize=(5, 4))

# scatter & dashed fit
ax.scatter(adj.index, y, marker='o', facecolors='none', edgecolors='black')
ax.plot(adj.index, y_fit, linestyle='--', color='black', linewidth=1.5)

# equation in bottom right
ax.text(
    0.98, 0.05, eqn,
    transform=ax.transAxes,
    ha='right', va='bottom',
    fontsize=7
)

# legend with R^2 only
legend_handle = Line2D([], [], linestyle='--', color='black')
ax.legend([legend_handle], [f"$R^2 = {r2:.2f}$"], loc='upper left')

# axes formatting
ax.set_xlabel('Publication Month')
ax.set_ylabel('Number of Studies')
ax.set_title('Monthly Study Counts')
ax.xaxis.set_major_locator(mdates.AutoDateLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
# save at 300 dpi as an svg
plt.savefig('monthly_study_counts_with_regression.svg', dpi=300)
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

df = pd.DataFrame(included_studies)
df['Date'] = pd.to_datetime(df['Date'])

tiers = ["all", 'I', 'II', 'III']
markers = ['o', 's', '^', 'D']
linestyles = ['-', '--', '-.', ':']

monthly_by_tier = {}
all_dates = []

for tier in tiers:
    if tier == "all":
        sub = df
    else:
        sub = df[df['LLM-tier'].apply(lambda d: d['Tier']==tier)]
    mc = (
        sub.assign(year_month=lambda x: x['Date'].dt.to_period('M').dt.to_timestamp())
           .groupby('year_month').size()
           .sort_index()
    )
    # redistribute January
    # the reason we must do this is that many studies that don't have a specific month are grouped into January
    adj = mc.copy()
    for ts, val in mc.items():
        if ts.month==1 and val>0:
            adj.loc[ts] = 0
            spread = val/12.0
            adj.loc[adj.index.year==ts.year] += spread

    monthly_by_tier[tier] = adj
    all_dates.extend(adj.index)

# compute global x-limits
x_min = min(all_dates)
x_max = max(all_dates)
global_max = max(adj.max() for adj in monthly_by_tier.values())

fig, axes = plt.subplots(2, 2, figsize=(10, 8), sharey=True)

for ax, tier, mk, ls in zip(axes.flat, tiers, markers, linestyles):
    adj = monthly_by_tier[tier]
    dates = adj.index
    nums  = mdates.date2num(dates)
    y     = adj.values

    # fit line
    m, b = np.polyfit(nums, y, 1)
    y_fit = m*nums + b
    slope_mo = m * 30.4375
    eqn = rf"$y = {slope_mo:.2f}\,x {b:+.1f}$"

    ax.scatter(dates, y, marker=mk, facecolors='none', edgecolors='black')
    ax.plot(dates, y_fit, linestyle=ls, color='black', linewidth=1.2)

    # set same x-limits on every subplot
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(0, global_max*1.1)

    if tier == "all":
        ax.set_title('All Tiers')
    else:
        ax.set_title(f'Tier {tier}')
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    ax.tick_params(axis='x', rotation=45)

    ax.text(
        0.95, 0.05, eqn,
        transform=ax.transAxes,
        ha='right', va='bottom',
        fontsize=7
    )

'''
for ax in axes[1]:
    ax.set_xlabel('Publication Month')
for ax in axes[:, 0]:
    ax.set_ylabel('Number of Studies')
'''
# shared labels
fig.supxlabel("Publication Month")
fig.supylabel("Number of Studies")

#fig.suptitle('Monthly Study Counts by Tier (January redistributed)', y=0.98)

plt.tight_layout(rect=[0,0,1,0.95])
plt.savefig('monthly_study_counts_by_tier.svg', dpi=300)
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.dates as mdates
import statsmodels.api as sm
from itertools import combinations
from scipy import stats
from statsmodels.stats.multitest import multipletests

df = pd.DataFrame(included_studies)
df['Date'] = pd.to_datetime(df['Date'])

tiers = ['S', 'I', 'II', 'III']
monthly_by_tier = {}

for tier in tiers:
    sub = df[df['LLM-tier'].apply(lambda d: d['Tier'] == tier)]
    mc = (
        sub.assign(year_month=lambda x: x['Date'].dt.to_period('M').dt.to_timestamp())
           .groupby('year_month').size()
           .sort_index()
    )
    # redistribute January bug
    adj = mc.copy()
    for ts, val in mc.items():
        if ts.month == 1 and val > 0:
            adj.loc[ts] = 0
            adj.loc[adj.index.year == ts.year] += val / 12.0
    monthly_by_tier[tier] = adj

# merge tier I and S due to small sample size of S
monthly_by_tier['I'] = monthly_by_tier['I'].add(monthly_by_tier['S'], fill_value=0)
del monthly_by_tier['S']
tiers.remove('S')

# Fit linear model per tier to extract slope & SE
results = {}
for tier, adj in monthly_by_tier.items():
    x_num = mdates.date2num(adj.index)
    X = sm.add_constant(x_num)
    model = sm.OLS(adj.values, X).fit()
    results[tier] = {
        'slope': model.params[1],
        'se': model.bse[1],
        'n_obs': int(model.nobs)
    }

# Pairwise slope‐difference t‐tests
comparisons = []
for t1, t2 in combinations(tiers, 2):
    b1, se1, n1 = results[t1]['slope'], results[t1]['se'], results[t1]['n_obs']
    b2, se2, n2 = results[t2]['slope'], results[t2]['se'], results[t2]['n_obs']
    t_stat = (b1 - b2) / np.sqrt(se1**2 + se2**2)
    df_num = (se1**2 + se2**2)**2
    df_den = (se1**4)/(n1-2) + (se2**4)/(n2-2)
    df_w = df_num / df_den
    p_raw = 2 * stats.t.sf(abs(t_stat), df_w)
    comparisons.append({
        'Comparison': f'Tier {t1} vs {t2}',
        't_stat': t_stat,
        'df': df_w,
        'p_value_raw': p_raw
    })

comp_df = pd.DataFrame(comparisons)

# Bonferroni correction
m = len(comp_df)
# manual
comp_df['p_bonf_manual'] = np.minimum(comp_df['p_value_raw'] * m, 1.0)
# using statsmodels
rej, p_adj, _, _ = multipletests(comp_df['p_value_raw'], alpha=0.05, method='bonferroni')
comp_df['p_bonf']    = p_adj
comp_df['reject_H0'] = rej

# Show results
comp_df[['Comparison', 't_stat', 'df', 'p_value_raw', 'p_bonf', 'reject_H0']]


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import multipletests

all_answers = []
not_found = 0
not_applicable = 0

records = []
for st in included_studies:
    ans = st.get('extracted_data', {}) \
            .get('did_the_llm_outperform_the_human?', {}) \
            .get('answer', '').lower()
    tier = st.get('LLM-tier', {}).get('Tier')
    all_answers.append(ans)
    if ("yes" in ans or ("no" in ans and "not" not in ans)) and tier in {'S', 'I', 'II', 'III'} and "pplicable" not in ans:
        records.append({'Tier': tier, 'Out': ans.capitalize().split()[0]})
    elif ("mixed" in ans or "partial" in ans) and tier in {'S', 'I', 'II', 'III'}:
        records.append({'Tier': tier, 'Out': 'Mixed/Partial'})
    elif ("n/a" in ans or "not applicable" in ans or "not_applicable" in ans) and tier in {'S', 'I', 'II', 'III'}:
        not_applicable += 1
    else:
        not_found += 1

df = pd.DataFrame(records)

# ── Compute summary proportions per Tier ────────────────────────────────
summary = df.groupby(['Tier','Out']).size().unstack(fill_value=0)
summary['Total'] = summary.sum(axis=1)
summary['Prop_Yes'] = summary['Yes'] / summary['Total']

# ── 1) Bar chart of proportion of Yes by Tier ──────────────────────────
fig, ax = plt.subplots(figsize=(3, 3))
tiers = ['I','II','III']
props = summary.loc[tiers, 'Prop_Yes'].values
ax.bar(tiers, props, color="lightgrey", edgecolor='black')
ax.set_xlabel('LLM Tier')
ax.set_title('Proportion of Studies Where LLM Outperformed Humans')
ax.yaxis.set_major_formatter(PercentFormatter(xmax=1))
ax.set_ylim(0, 0.4)
for i, prop in enumerate(props):
    ax.text(i, prop + 0.02, f"{prop:.1%}", ha='center')
plt.tight_layout()
plt.savefig("proportion_yes_by_tier.svg")
plt.show()

# ── 2) One-sided z-tests with Bonferroni correction ─────────────────────
comparisons = [('I','II'), ('I','III'), ('II','III')]
p_vals = []
labels = []
for a,b in comparisons:
    count = np.array([int(summary.loc[a, 'Yes']), int(summary.loc[b, 'Yes'])])
    nobs = np.array([int(summary.loc[a, 'Total']), int(summary.loc[b, 'Total'])])
    stat, p = proportions_ztest(count, nobs, alternative='smaller')
    p_vals.append(p)
    labels.append(f"{a} < {b}")

# Correct for multiple comparisons
rejected, p_adj, _, _ = multipletests(p_vals, alpha=0.05, method='bonferroni')

# Display results
results = pd.DataFrame({
    'Comparison': labels,
    'Raw p-value': p_vals,
    'Adj. p-value': p_adj,
    'Significant': rejected
})
print("\nOne-Sided Pairwise Comparisons (Bonferroni-corrected):")
print(results.to_string(index=False))

# ── 1) Bar chart of proportion of Yes by Tier ──────────────────────────
fig, ax = plt.subplots(figsize=(5, 5))
tiers = ['I','II','III']
props = summary.loc[tiers, 'Prop_Yes'].values
bars = ax.bar(tiers, props, color="lightgrey", edgecolor='black')
ax.set_xlabel('LLM Tier')
ax.set_title('Proportion of Studies Where LLM Outperformed Humans')
ax.yaxis.set_major_formatter(PercentFormatter(xmax=1))
ax.set_ylim(0, 0.4)
for i, prop in enumerate(props):
    ax.text(i, prop + 0.02, f"{prop:.1%}", ha='center')

# ── 2) One‐sided z‐tests with Bonferroni correction ────────────────────
comparisons = [('I','II'), ('I','III'), ('II','III')]
p_vals = []
for a,b in comparisons:
    count = np.array([summary.loc[a, 'Yes'], summary.loc[b, 'Yes']])
    nobs  = np.array([summary.loc[a, 'Total'], summary.loc[b, 'Total']])
    _, p = proportions_ztest(count, nobs, alternative='smaller')
    p_vals.append(p)

rejected, p_adj, _, _ = multipletests(p_vals, alpha=0.05, method='bonferroni')

# ── 3) Draw significance bars ──────────────────────────────────────────
def add_sig_bar(ax, x1, x2, y, h, text):
    """Draw a horizontal bar with vertical ticks at x1,x2 and place text above."""
    ax.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c='black')
    ax.text((x1 + x2)*0.5, y + h, text, ha='center', va='bottom')

# compute where to start the first bar
y_max = props.max()
bar_offset = 0.03
line_height = 0.015

for i, ((a,b), sig) in enumerate(zip(comparisons, rejected)):
    x1 = tiers.index(a)
    x2 = tiers.index(b)
    y = y_max + bar_offset + i*(line_height*2)
    label = '*' if sig else 'n.s.'
    add_sig_bar(ax, x1, x2, y, line_height, label)

# extend y‐limit if needed
ax.set_ylim(0, y + line_height*2)

plt.tight_layout()
plt.savefig("proportion_yes_by_tier_with_significance.svg", dpi=300)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import multipletests
from itertools import combinations  # ← added

# Prepare data per year
records = []
for st in included_studies:
    date = st.get('Date')
    if date is None:
        continue
    year = pd.to_datetime(date).year
    if year == 2022:
        continue

    tier = st.get('LLM-tier', {}).get('Tier')
    ans = st.get('extracted_data', {})\
            .get('did_the_llm_outperform_the_human?', {})\
            .get('answer', '').lower()
    if tier == "S":
        tier = "I"
    if 'yes' in ans or ('no' in ans and 'not' not in ans):
        records.append({'Year': year, 'Tier': tier, 'Out': ans.capitalize()})
    elif 'mixed' in ans or 'partial' in ans:
        # split mixed/partial counts equally
        records.append({'Year': year, 'Tier': tier, 'Out': 'Yes'})
        records.append({'Year': year, 'Tier': tier, 'Out': 'No'})

df_year = pd.DataFrame(records)

# ── Compute counts and proportions ───────────────────────────────────────
counts = df_year.groupby(['Year', 'Tier', 'Out']).size().unstack(fill_value=0)
totals = counts.sum(axis=1)
prop_yes = (counts['Yes'] / totals).unstack(level='Tier').loc[:, ['I','II','III']]

# ── Total sample size per year ──────────────────────────────────────────
year_totals = df_year.groupby('Year').size()

# ── Plot grouped bars by year with black/grey/hatched styling ──────────
years = prop_yes.index.values
x = np.arange(len(years))
width = 0.25

styles = [
    {'color': 'black',      'hatch': None},   # Tier I
    {'color': 'lightgrey',  'hatch': None},   # Tier II
    {'color': 'white',      'hatch': '///'}   # Tier III
]

fig, ax = plt.subplots(figsize=(6, 4))
for i, tier in enumerate(['I','II','III']):
    ax.bar(
        x + i*width,
        prop_yes[tier],
        width,
        label=f'Tier {tier}',
        color=styles[i]['color'],
        hatch=styles[i]['hatch'],
        edgecolor='black'
    )

# annotate sample size above each cluster
for i, year in enumerate(years):
    ax.text(
        x[i] + width,
        max(prop_yes.loc[year]) + 0.02,
        f"n={year_totals.loc[year]}",
        ha='center', va='bottom', fontsize=8
    )

ax.set_xlabel('Publication Year')
ax.set_title('Yearly LLM Outperformance by Tier')
ax.set_xticks(x + width)
ax.set_xticklabels(years)
ax.yaxis.set_major_formatter(PercentFormatter(1))
ax.set_ylim(0, 0.5)
ax.legend(loc='upper left', bbox_to_anchor=(0, 1.15))

plt.tight_layout()
plt.savefig("yearly_outperformance_by_tier.svg", dpi=300)
plt.show()

# One-sided z-tests within each year
comparisons = [('I','II'), ('I','III'), ('II','III')]
print("One-sided proportion tests (Tier A < Tier B) per Year:\n")
for year in years:
    y_counts = counts.loc[year]
    yes_counts = y_counts['Yes']
    total_counts = y_counts.sum(axis=1)

    p_vals = []
    labels = []
    for a, b in comparisons:
        count = np.array([yes_counts[a], yes_counts[b]])
        nobs = np.array([total_counts[a], total_counts[b]])
        stat, p = proportions_ztest(count, nobs, alternative='smaller')
        p_vals.append(p)
        labels.append(f"{a} < {b}")

    # Bonferroni correction
    rejected, p_adj, _, _ = multipletests(p_vals, alpha=0.05, method='bonferroni')

    print(f"Year {year}:")
    for lbl, raw_p, adj_p, sig in zip(labels, p_vals, p_adj, rejected):
        star = "✓" if sig else "✗"
        print(f"  {lbl}: raw p={raw_p:.3f}, adj p={adj_p:.3f} {star}")
    print()

# Inter-year comparisons for each Tier
print("Inter-year proportion tests (two-sided) for each Tier:\n")
for tier in ['I', 'II', 'III']:
    # Extract the Yes counts and totals for this tier across years
    yes_counts_t = counts.xs(tier, level='Tier')['Yes']
    total_counts_t = counts.xs(tier, level='Tier').sum(axis=1)

    # All unique year-pairs
    year_pairs = list(combinations(years, 2))
    p_vals = []
    labels = []

    for y1, y2 in year_pairs:
        count = np.array([yes_counts_t[y1], yes_counts_t[y2]])
        nobs  = np.array([total_counts_t[y1], total_counts_t[y2]])
        stat, p = proportions_ztest(count, nobs, alternative='two-sided')
        p_vals.append(p)
        labels.append(f"{y1} vs {y2}")

    # Bonferroni correction across this tier’s comparisons
    rejected, p_adj, _, _ = multipletests(p_vals, alpha=0.05, method='bonferroni')

    print(f"Tier {tier}:")
    for lbl, raw_p, adj_p, sig in zip(labels, p_vals, p_adj, rejected):
        star = "✓" if sig else "✗"
        print(f"  {lbl}: raw p={raw_p:.3f}, adj p={adj_p:.3f} {star}")
    print()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def classify_evaluator(role: str) -> str:
    """
    Classify evaluator into one of 10 broad categories
    (Attending, MD Unspecified, Resident, Fellow, Medical Student,
    Student (Other), Researcher/Academic, Non-Physician Clinician,
    Patient/Public, Other).
    """

    role = role.strip().lower()

    # --- Attending ---
    attending_roles = {"attending", "attending (fellowship-trained)"}

    # --- MD unspecified ---
    md_unspecified = {"md (unspecified)", "physician (md/do unspecified)"}

    # --- Resident ---
    resident_roles = {"resident"}

    # --- Fellow ---
    fellow_roles = {"fellow"}

    # --- Medical Student ---
    med_student_roles = {"medical student", "sub-intern"}

    # --- Student (Other) ---
    student_other_roles = {
        "student (graduate, not medical)",
        "student (undergraduate)",
        "student (other)",
        "intern",           # often lumped here in high-level grouping
        "other trainee"
    }

    # --- Researcher / Academic ---
    academic_roles = {
        "researcher / scientist", "faculty / educator", "other academic"
    }

    # --- Non-Physician Clinician ---
    clinician_roles = {
        "nurse", "advanced practice provider (np/pa)",
        "therapist / rehabilitation / mental health",
        "pharmacist", "technician / emt", "other clinical provider"
    }

    # --- Patient / Public ---
    patient_public_roles = {
        "lay public / participant",
        "caregiver / family member",
        "non-healthcare domain expert"
    }

    # --- Other ---
    other_roles = {
        "reviewer / panelist", "other support / admin", "language services",
        "information specialist", "medical coder", "other", "other (public)"
    }

    # Classification logic
    if role in attending_roles:
        return "Attending"
    elif role in md_unspecified:
        return "MD (Unspecified)"
    elif role in resident_roles:
        return "Resident"
    elif role in fellow_roles:
        return "Fellow"
    elif role in med_student_roles:
        return "Medical Student"
    elif role in student_other_roles:
        return "Student (Other)"
    elif role in academic_roles:
        return "Researcher / Academic"
    elif role in clinician_roles:
        return "Non-Physician Clinician"
    elif role in patient_public_roles:
        return "Patient / Public"
    elif role in other_roles:
        return "Other"
    else:
        return "Other"



all_others = []
buckets = []
found = 0
for st in included_studies:
    prev_len = len(buckets)
    for raw in st.get('processed_data', {}).get('human_evaluators', []):
        category = classify_evaluator(raw)
        if category == 'Other':
            all_others.append(raw)
        if category != "Unsure":
            buckets.append(category)
    
    if len(buckets) > prev_len:
        found += 1

counts = pd.Series(buckets).value_counts().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(6, 4))
ax.barh(counts.index, counts.values, color='lightgrey', edgecolor='black')
ax.set_xlabel('Number of Evaluator Entries')
ax.set_title('Human Evaluator Composition with MD (Unspecified) Category')

for y, v in enumerate(counts.values):
    ax.text(v + 50, y, f"{v}, {v/len(buckets)*100:.1f}%", va='center')

plt.tight_layout()
plt.savefig("human_evaluator_composition.svg", dpi=300)
plt.show()

md_roles = [
    "Attending",
    "Attending (Fellowship-trained)",
    "Fellow",
    "Intern",
    "MD (Unspecified)",
    "Physician (MD/DO unspecified)",
    "Resident",
]

# get number that compared to a medical doctor period
md_count = sum(1 for cat in buckets if cat in md_roles)
#print(md_count, md_count / len(buckets))
print("Number of MD comparisons:", md_count)
print("Percentage of MD comparisons: {:.1f}%".format(md_count / len(buckets) * 100))

# now, get the % of each MD category that makes up the total MD comparisons, also print the absolute number
md_categories = ['Attending', 'MD (Unspecified)', 'Resident', 'Fellow']
md_counts = counts.loc[md_categories]
md_total = md_counts.sum()
for cat in md_categories:
    cnt = md_counts.loc[cat]
    print(f"{cat}: {cnt} ({cnt/md_total*100:.1f}%)")

In [ ]:
md_roles = [
    "Attending",
    "Attending (Fellowship-trained)",
    "Fellow",
    "Intern",
    "MD (Unspecified)",
    "Physician (MD/DO unspecified)",
    "Resident",
]

general_public = [
    "Caregiver / Family Member",
    "Lay Public / Participant",
    "Other (Public)",
    "Patient"
]

set_of_comparators = general_public

set_of_comparators = [item.lower().strip() for item in set_of_comparators]

all_counts = {} # The number of comparisons found for each category
counts = {} # The number of outcomes found for each category (some comparisons do not have outcomes)

outcomes_found = 0
for st in included_studies:
    ans_raw = st.get('extracted_data', {}) \
        .get('did_the_llm_outperform_the_human?', {}) \
        .get('answer', '').lower()
    
    if ans_raw not in ("", "n/a", "not applicable", "not_applicable", "unsure"):
        outcomes_found += 1
        comparators = [item.lower().strip() for item in st["processed_data"]["human_evaluators"]]

        if any(role in set_of_comparators for role in comparators):
            for role in comparators:
                if role in set_of_comparators:
                    counts[role] = counts.get(role, 0) + 1
        else:
            counts["other"] = counts.get("other", 0) + 1
    
    for item in st["processed_data"]["human_evaluators"]:
        if item.lower().strip() in set_of_comparators:
            all_counts[item] = all_counts.get(item, 0) + 1
        else:
            all_counts["other"] = all_counts.get("other", 0) + 1

print(f"Found outcomes for {outcomes_found} studies.")
print(all_counts)

In [ ]:
results_found = 0
outcomes = []
for st in included_studies:
    ans_raw = st.get('extracted_data', {}) \
        .get('did_the_llm_outperform_the_human?', {}) \
        .get('answer', '').lower()
    
    if ans_raw not in ("", "n/a", "not applicable", "not_applicable", "unsure"):
        comparators = [item.lower().strip() for item in st["processed_data"]["human_evaluators"]]
        results_found += 1
    
    if 'mixed' in ans_raw or 'partial' in ans_raw:
        outcomes.append("mixed")
    elif 'yes' in ans_raw:
        outcomes.append("yes")
    elif 'no' in ans_raw and "not" not in ans_raw:
        outcomes.append("no")
    #else: # uncomment this to show the full set of raw answers, which is: {unsure, no, yes, mixed, not applicable, not_applicable, n/a}
    #    outcomes.append(ans_raw)

# get counts of each
results = {}
for item in outcomes:
    results[item] = results.get(item, 0) + 1

# print results, both absolute and percentage
total = sum(results.values())
for key, val in results.items():
    print(f"{key}: {val} ({val/total*100:.1f}%)")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import multipletests
import itertools

# Collect outcome data across included tiers
records = []
for st in included_studies:
    tier = st.get('LLM-tier', {}).get('Tier')
    if tier not in {'I', 'II', 'III'}:
        continue

    ans_raw = st.get('extracted_data', {}) \
                .get('did_the_llm_outperform_the_human?', {}) \
                .get('answer', '').lower()

    if 'mixed' in ans_raw or 'partial' in ans_raw:
        outs = ['Yes', 'No']
    elif 'yes' in ans_raw:
        outs = ['Yes']
    elif 'no' in ans_raw and 'not' not in ans_raw:
        outs = ['No']
    else:
        continue

    for cat in st.get('processed_data', {}).get('human_evaluators', []):
        if cat == "Attending (Fellowship-trained)":
            cat = "Attending"
        if cat == "Intern":
            cat = "Resident"
        if cat == "Sub-intern":
            cat = "Medical Student"
        if (
            cat == "Advanced Practice Provider (NP/PA)" or
            cat == "Nurse" or
            cat == "Pharmacist" or
            cat == "Technician / EMT" or
            cat == "Therapist / Rehabilitation / Mental Health" or
            cat == "Other Clinical Provider"
        ):
            cat = "Non-physician Clinician"

        for out in outs:
            records.append({'Category': cat, 'Out': out})

df = pd.DataFrame(records)

# ── 2) Compute proportions of “Yes” per category ─────────────────────────
summary = df.groupby(['Category', 'Out']).size().unstack(fill_value=0)
summary['Total'] = summary.sum(axis=1)
summary['Prop_Yes'] = summary['Yes'] / summary['Total']

# order categories
order = [
    'Non-physician Clinician',
    'Attending',
    "Resident",
    "Medical Student",
    'MD (Unspecified)',
]
summary = summary.loc[order]

# ── helper to draw significance bars ─────────────────────────────────────
def add_sig_bar(ax, x1, x2, y, h, text):
    ax.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c='black')
    ax.text((x1 + x2) / 2, y + h, text, ha='center', va='bottom')

# ── 3) Plot bar chart ────────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(5, 5))
bars = ax.bar(summary.index, summary['Prop_Yes'],
              color='lightgrey', edgecolor='black')

ax.yaxis.set_major_formatter(PercentFormatter(1))
ax.set_ylim(0, 0.6)

# annotate percentages and sample sizes
for i, (cat, row) in enumerate(summary.iterrows()):
    prop  = row['Prop_Yes']
    total = int(row['Total'])
    ax.text(i, prop + 0.02, f"{prop:.0%}", ha='center')
    ax.text(i, 0.75,      f"n={total}",      ha='center',
            va='bottom', fontsize=8)

plt.xticks(rotation=45, ha='right')

# Pairwise z‐tests & FDR‐BH correction
categories = summary.index.tolist()
pairs      = list(itertools.permutations(categories, 2))

p_vals = []
labels = []
for a, b in pairs:
    count = [summary.loc[a, 'Yes'], summary.loc[b, 'Yes']]
    nobs  = [summary.loc[a, 'Total'], summary.loc[b, 'Total']]
    _, p = proportions_ztest(count, nobs, alternative='larger')
    p_vals.append(p)
    labels.append(f"{a} > {b}")

# use FDR-BH to match your significance table
rejected, p_adj, _, _ = multipletests(p_vals, alpha=0.05, method='fdr_gbs')

results   = pd.DataFrame({
    'Comparison':   labels,
    'Adj. p-value': p_adj,
    'Significant':  rejected
})
sig_comps = results.loc[results['Significant'], 'Comparison'].tolist()

# Draw significance bars for the significant comparisons
y_max      = summary['Prop_Yes'].max()
bar_offset = 0.05
line_h     = 0.015

for i, comp in enumerate(sig_comps):
    a, b = comp.split(' > ')
    x1 = categories.index(b)
    x2 = categories.index(a)
    y  = y_max + bar_offset + i * (line_h * 2)
    add_sig_bar(ax, x1, x2, y, line_h, '*')

# extend y‐limit to fit all bars
ax.set_ylim(0, y + line_h * 2)

plt.tight_layout()
plt.savefig("llm_outperformance_by_evaluator_with_significance.svg", dpi=300)
plt.show()

In [ ]:
import pandas as pd
import itertools
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import multipletests

categories = summary.index.tolist()

# Prepare pairwise tests for all ordered pairs
pairs = list(itertools.permutations(categories, 2))
p_vals = []
labels = []

for a, b in pairs:
    count = [int(summary.loc[a, 'Yes']), int(summary.loc[b, 'Yes'])]
    nobs = [int(summary.loc[a, 'Total']), int(summary.loc[b, 'Total'])]
    stat, p = proportions_ztest(count, nobs, alternative='larger')
    p_vals.append(p)
    labels.append(f"{a} > {b}")

# Apply correction
rejected, p_adj, _, _ = multipletests(p_vals, alpha=0.05, method='bonferroni')

# Compile and display results
results = pd.DataFrame({
    'Comparison': labels,
    'Raw p-value': p_vals,
    'Adj. p-value': p_adj,
    'Significant': rejected
})
print(results.query("Significant").to_string(index=False))

In [ ]:
# print all results where the raw p-value is less than 0.05
print("\nAll comparisons with raw p-value < 0.05:\n")
print(results.query("`Raw p-value` < 0.05").to_string(index=False))